In [ ]:
import numpy as np
import random

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader 

In [ ]:
# Set up for getting reproducibility of result
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
data_dir = '/content/drive/MyDrive/Datasets/Oxford Flower'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Define transformation
transforms = transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                                      [0.229, 0.224, 0.225])
])

# Load the datsets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=transforms)

# Define DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32)
test_loader = DataLoader(dataset=test_dataset, batch_size=32)

In [ ]:
# Load VGG-16 model
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Freeze pretrained model parameters
for parameter in model.parameters():
  parameter.requires_grad = False

# Build custom classifier
classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 102),
    nn.LogSoftmax(dim=1)
)

model.classifier = classifier
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [ ]:
def validation(model, valid_loader, criterion):
    val_loss = 0
    accuracy = 0

    for images, labels in iter(valid_loader):

        images = images.to(device)
        labels = labels.to(device)

        output = model(images)

        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)

        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    return val_loss, accuracy

In [ ]:
import time

def train_model():
    max_epoch = 10
    print_every = 20
    n_steps = 0
    model.to(device)

    start_train = time.time()
    for n_epochs in range(max_epoch):
        print(f' - Epoch {n_epochs+1}')
        start_epoch = time.time()
        model.train()

        epoch_loss = 0

        for batch_idx, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model.forward(images)

            loss = criterion(output, labels)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

            if n_steps % print_every == 0:
                model.eval()

                with torch.no_grad():
                    valid_loss, accuracy = validation(model, valid_loader, criterion)

                print(f'Batch: {batch_idx}/{len(train_loader)}..  '
                         f'Training Loss: {epoch_loss/print_every:.3f}..  ',
                         f'Validation Loss: {valid_loss/len(valid_loader):.3f}..  ',
                         f'Validation Accuracy: {accuracy/len(valid_loader):.3f}')

                epoch_loss = 0
                model.train()
            n_steps += 1
        end_epoch = time.time()
        print(f'--> Epoch {n_epochs + 1} costs {end_epoch - start_epoch} seconds')
    end_train = time.time()
    print(f'Training time: {end_train - start_train}')


In [ ]:
train_model()

 - Epoch 1
Batch: 0/103..  Training Loss: 0.076..   Validation Loss: 1.343..   Validation Accuracy: 0.641
Batch: 20/103..  Training Loss: 1.462..   Validation Loss: 1.051..   Validation Accuracy: 0.733
Batch: 40/103..  Training Loss: 1.401..   Validation Loss: 1.021..   Validation Accuracy: 0.736
Batch: 60/103..  Training Loss: 1.366..   Validation Loss: 0.832..   Validation Accuracy: 0.778
Batch: 80/103..  Training Loss: 1.292..   Validation Loss: 0.864..   Validation Accuracy: 0.782
Batch: 100/103..  Training Loss: 1.361..   Validation Loss: 0.879..   Validation Accuracy: 0.748
--> Epoch 1 costs 889.367359161377 seconds
 - Epoch 2
Batch: 17/103..  Training Loss: 0.708..   Validation Loss: 0.788..   Validation Accuracy: 0.795
Batch: 37/103..  Training Loss: 0.782..   Validation Loss: 0.851..   Validation Accuracy: 0.779
Batch: 57/103..  Training Loss: 0.884..   Validation Loss: 0.879..   Validation Accuracy: 0.767
Batch: 77/103..  Training Loss: 0.887..   Validation Loss: 0.742..   Va

In [ ]:
def save_checkpoint(model):

    model.class_to_idx = train_dataset.class_to_idx

    checkpoint = {'arch': 'vgg16',
                            'class_to_idx': model.class_to_idx,
                            'model_state_dict': model.state_dict()
    }
    torch.save(checkpoint, '/content/drive/MyDrive/Colab Notebooks/Assignment/checkpoint.pth')

In [ ]:
save_checkpoint(model)

In [ ]:
def test_model(model, test_loader):
    model.eval()
    model.to(device)

    with torch.no_grad():
        accuracy = 0
        for images, labels in iter(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            output = model(images)

           

            probabilities = torch.exp(output)

            equality = (labels.data == probabilities.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
        print(f'Test Accuracy: {accuracy/len(test_loader)}')

In [ ]:
test_model(model, test_loader)

Test Accuracy: 0.7823253870010376


In [ ]:
!nvidia-smi

Tue Dec 14 04:12:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    73W / 149W |   5024MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------